In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import re

In [4]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG-Python

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 8.2MB 4.5MB/s 
     |████████████████████████████████| 7.1MB 2.7MB/s 


In [3]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Читаем. Смотрим

In [4]:
df = pd.read_csv("/content/drive/My Drive/Datasets_LCT/events.csv")

In [5]:
# data = pd.read_csv("/content/drive/My Drive/dataset/dirty/events.csv")

In [6]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Тип учреждения,Подразделение,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Мероприятие относится к ежегодным праздникам и общегородским мероприятиям,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Целевая аудитория
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,Дома культуры,NaN,бесплатное,фестиваль,культурно-досуговое,NaN,NaN,13.06.2020 16:00,NaN,13.06.2020 20:00,NaN,Клуб Современник проведет фестиваль современны...,Нет,NaN,NaN,NaN,недоступно,NaN,0+,от 20 до 40,NaN
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,NaN,бесплатное,"танцевальный вечер, бал",патриотическое,праздник,9 мая. День Победы,09.05.2020 14:30,NaN,09.05.2020 17:30,NaN,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,NaN,16+,от 40 до 85,"работающая молодежь, взрослые, пенсионеры"
2,3179,Врасплох,Завершено,ГБУК г. Москвы «ДК «Нагатино»,Дома культуры,NaN,бесплатное,выставка,антитеррор,NaN,NaN,01.01.2020,9:00,31.01.2020,22:00,Регулярное (ежеквартальное) мероприятие цикла ...,Нет,"Судостроительная ул. 31, корпус 1",Южный административный округ,Нагатинский Затон,NaN,NaN,6+,от 16 до 100,"школьники, работающая молодежь, студенческая м..."
3,486,Праздник Новогодней елки. Новогодний концерт в...,Завершено,ГАУК г. Москвы «Культурный центр ЗИЛ»,Дома культуры,NaN,бесплатное,"концерт, музыкальное представление",культурно-досуговое,праздник,1 января. Новый год,01.01.2020,10:00,01.01.2020,19:00,Коллектив Культурного центра ЗИЛ Ансамбль эстр...,Нет,"Россия, Москва",NaN,NaN,NaN,NaN,0+,от 5 до 14,"дети дошкольного возраста, школьники"
4,2100,"Выставка к Рождеству ""Сувенир Года""",Завершено,ГБУК г.Москвы ТКС «Орехово»,Дома культуры,NaN,бесплатное,выставка,культурно-досуговое,праздник,1 января. Новый год,01.01.2020,10:00,07.01.2020,20:00,В преддверии волшебного праздника Рождества Хр...,Нет,"Борисовский пр. 22, строение 2, корпус 1",Южный административный округ,Орехово-Борисово Северное,недоступно,NaN,6+,от 15 до 80,"дети дошкольного возраста, школьники, работающ..."


# Дропаем ненужное

In [7]:
df = df.drop('Подразделение', 1)

In [8]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Тип учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Мероприятие относится к ежегодным праздникам и общегородским мероприятиям,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Целевая аудитория
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,Дома культуры,бесплатное,фестиваль,культурно-досуговое,NaN,NaN,13.06.2020 16:00,NaN,13.06.2020 20:00,NaN,Клуб Современник проведет фестиваль современны...,Нет,NaN,NaN,NaN,недоступно,NaN,0+,от 20 до 40,NaN
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,бесплатное,"танцевальный вечер, бал",патриотическое,праздник,9 мая. День Победы,09.05.2020 14:30,NaN,09.05.2020 17:30,NaN,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,NaN,16+,от 40 до 85,"работающая молодежь, взрослые, пенсионеры"
2,3179,Врасплох,Завершено,ГБУК г. Москвы «ДК «Нагатино»,Дома культуры,бесплатное,выставка,антитеррор,NaN,NaN,01.01.2020,9:00,31.01.2020,22:00,Регулярное (ежеквартальное) мероприятие цикла ...,Нет,"Судостроительная ул. 31, корпус 1",Южный административный округ,Нагатинский Затон,NaN,NaN,6+,от 16 до 100,"школьники, работающая молодежь, студенческая м..."
3,486,Праздник Новогодней елки. Новогодний концерт в...,Завершено,ГАУК г. Москвы «Культурный центр ЗИЛ»,Дома культуры,бесплатное,"концерт, музыкальное представление",культурно-досуговое,праздник,1 января. Новый год,01.01.2020,10:00,01.01.2020,19:00,Коллектив Культурного центра ЗИЛ Ансамбль эстр...,Нет,"Россия, Москва",NaN,NaN,NaN,NaN,0+,от 5 до 14,"дети дошкольного возраста, школьники"
4,2100,"Выставка к Рождеству ""Сувенир Года""",Завершено,ГБУК г.Москвы ТКС «Орехово»,Дома культуры,бесплатное,выставка,культурно-досуговое,праздник,1 января. Новый год,01.01.2020,10:00,07.01.2020,20:00,В преддверии волшебного праздника Рождества Хр...,Нет,"Борисовский пр. 22, строение 2, корпус 1",Южный административный округ,Орехово-Борисово Северное,недоступно,NaN,6+,от 15 до 80,"дети дошкольного возраста, школьники, работающ..."


In [9]:
df = df.drop('Тип учреждения', 1)

In [10]:
df = df.drop('Мероприятие относится к ежегодным праздникам и общегородским мероприятиям', 1)

In [11]:
df = df.drop('Целевая аудитория', 1)

In [12]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,бесплатное,фестиваль,культурно-досуговое,NaN,13.06.2020 16:00,NaN,13.06.2020 20:00,NaN,Клуб Современник проведет фестиваль современны...,Нет,NaN,NaN,NaN,недоступно,NaN,0+,от 20 до 40
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,бесплатное,"танцевальный вечер, бал",патриотическое,9 мая. День Победы,09.05.2020 14:30,NaN,09.05.2020 17:30,NaN,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,NaN,16+,от 40 до 85
2,3179,Врасплох,Завершено,ГБУК г. Москвы «ДК «Нагатино»,бесплатное,выставка,антитеррор,NaN,01.01.2020,9:00,31.01.2020,22:00,Регулярное (ежеквартальное) мероприятие цикла ...,Нет,"Судостроительная ул. 31, корпус 1",Южный административный округ,Нагатинский Затон,NaN,NaN,6+,от 16 до 100
3,486,Праздник Новогодней елки. Новогодний концерт в...,Завершено,ГАУК г. Москвы «Культурный центр ЗИЛ»,бесплатное,"концерт, музыкальное представление",культурно-досуговое,1 января. Новый год,01.01.2020,10:00,01.01.2020,19:00,Коллектив Культурного центра ЗИЛ Ансамбль эстр...,Нет,"Россия, Москва",NaN,NaN,NaN,NaN,0+,от 5 до 14
4,2100,"Выставка к Рождеству ""Сувенир Года""",Завершено,ГБУК г.Москвы ТКС «Орехово»,бесплатное,выставка,культурно-досуговое,1 января. Новый год,01.01.2020,10:00,07.01.2020,20:00,В преддверии волшебного праздника Рождества Хр...,Нет,"Борисовский пр. 22, строение 2, корпус 1",Южный административный округ,Орехово-Борисово Северное,недоступно,NaN,6+,от 15 до 80


# Стоимость полная

Преобразуем в 3 столбца:

1) Стоимость:

    0 - бесплатно
    1 - платно

2) Стоимость ОТ

3) Стоимость ДО

In [13]:
df['Стоимость (полная)'].unique()

array(['бесплатное', 'платное (от 150 руб. до 150 руб.)', nan,
       'платное (от 700 руб. до 3400 руб.)',
       'платное (от 900 руб. до 1800 руб.)',
       'платное (от 600 руб. до 1200 руб.)',
       'платное (от 700 руб. до 1300 руб.)',
       'платное (от 1000 руб. до 1000 руб.)',
       'платное (от 500 руб. до 750 руб.)',
       'платное (от 600 руб. до 2500 руб.)',
       'платное (от 2950 руб. до 2950 руб.)',
       'платное (от 700 руб. до 700 руб.)',
       'платное (от 750 руб. до 4000 руб.)',
       'платное (от 1200 руб. до 2500 руб.)',
       'платное (от 750 руб. до 5000 руб.)',
       'платное (от 1200 руб. до 3500 руб.)',
       'платное (от 1000 руб. до 6000 руб.)',
       'платное (от 70 руб. до 70 руб.)',
       'платное (от 650 руб. до 800 руб.)',
       'платное (от 5000 руб. до 5000 руб.)',
       'платное (от 400 руб. до 600 руб.)',
       'платное (от 600 руб. до 1600 руб.)',
       'платное (от 500 руб. до 2000 руб.)',
       'платное (от 300 руб. до 300 ру

In [14]:
df['Стоимость'] = '1'

In [15]:
df['Стоимость (полная)'].describe()

count          39606
unique           219
top       бесплатное
freq           38084
Name: Стоимость (полная), dtype: object

In [16]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Стоимость
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,бесплатное,фестиваль,культурно-досуговое,NaN,13.06.2020 16:00,NaN,13.06.2020 20:00,NaN,Клуб Современник проведет фестиваль современны...,Нет,NaN,NaN,NaN,недоступно,NaN,0+,от 20 до 40,1
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,бесплатное,"танцевальный вечер, бал",патриотическое,9 мая. День Победы,09.05.2020 14:30,NaN,09.05.2020 17:30,NaN,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,NaN,16+,от 40 до 85,1
2,3179,Врасплох,Завершено,ГБУК г. Москвы «ДК «Нагатино»,бесплатное,выставка,антитеррор,NaN,01.01.2020,9:00,31.01.2020,22:00,Регулярное (ежеквартальное) мероприятие цикла ...,Нет,"Судостроительная ул. 31, корпус 1",Южный административный округ,Нагатинский Затон,NaN,NaN,6+,от 16 до 100,1
3,486,Праздник Новогодней елки. Новогодний концерт в...,Завершено,ГАУК г. Москвы «Культурный центр ЗИЛ»,бесплатное,"концерт, музыкальное представление",культурно-досуговое,1 января. Новый год,01.01.2020,10:00,01.01.2020,19:00,Коллектив Культурного центра ЗИЛ Ансамбль эстр...,Нет,"Россия, Москва",NaN,NaN,NaN,NaN,0+,от 5 до 14,1
4,2100,"Выставка к Рождеству ""Сувенир Года""",Завершено,ГБУК г.Москвы ТКС «Орехово»,бесплатное,выставка,культурно-досуговое,1 января. Новый год,01.01.2020,10:00,07.01.2020,20:00,В преддверии волшебного праздника Рождества Хр...,Нет,"Борисовский пр. 22, строение 2, корпус 1",Южный административный округ,Орехово-Борисово Северное,недоступно,NaN,6+,от 15 до 80,1


In [17]:
df.loc[df['Стоимость (полная)'] == 'бесплатное', 'Стоимость'] = '0'

df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Стоимость
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,бесплатное,фестиваль,культурно-досуговое,NaN,13.06.2020 16:00,NaN,13.06.2020 20:00,NaN,Клуб Современник проведет фестиваль современны...,Нет,NaN,NaN,NaN,недоступно,NaN,0+,от 20 до 40,0
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,бесплатное,"танцевальный вечер, бал",патриотическое,9 мая. День Победы,09.05.2020 14:30,NaN,09.05.2020 17:30,NaN,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,NaN,16+,от 40 до 85,0
2,3179,Врасплох,Завершено,ГБУК г. Москвы «ДК «Нагатино»,бесплатное,выставка,антитеррор,NaN,01.01.2020,9:00,31.01.2020,22:00,Регулярное (ежеквартальное) мероприятие цикла ...,Нет,"Судостроительная ул. 31, корпус 1",Южный административный округ,Нагатинский Затон,NaN,NaN,6+,от 16 до 100,0
3,486,Праздник Новогодней елки. Новогодний концерт в...,Завершено,ГАУК г. Москвы «Культурный центр ЗИЛ»,бесплатное,"концерт, музыкальное представление",культурно-досуговое,1 января. Новый год,01.01.2020,10:00,01.01.2020,19:00,Коллектив Культурного центра ЗИЛ Ансамбль эстр...,Нет,"Россия, Москва",NaN,NaN,NaN,NaN,0+,от 5 до 14,0
4,2100,"Выставка к Рождеству ""Сувенир Года""",Завершено,ГБУК г.Москвы ТКС «Орехово»,бесплатное,выставка,культурно-досуговое,1 января. Новый год,01.01.2020,10:00,07.01.2020,20:00,В преддверии волшебного праздника Рождества Хр...,Нет,"Борисовский пр. 22, строение 2, корпус 1",Южный административный округ,Орехово-Борисово Северное,недоступно,NaN,6+,от 15 до 80,0


In [18]:
df[df['Стоимость'] == '1'].count()

id                                        2990
Название мероприятия                      2990
Статус                                    2990
Наименование учреждения                   2990
Стоимость (полная)                        1522
Тип мероприятия                           2989
Направленность мероприятия                2921
Наименование праздника или события         365
Дата начала мероприятия                   2990
Время начала мероприятия                  2402
Дата окончания мероприятия                2990
Время окончания мероприятия               2402
Краткое описание                          2990
Проводится онлайн                         2990
Место проведения                          1968
Округ                                     1881
Район                                     1871
Доступность мероприятия для лиц с ОВЗ     2008
Тип ОВЗ                                    905
Возрастной ценз участников мероприятия    2990
Возрастная категория                      2990
Стоимость    

In [19]:
def get_lists(x, d=0):
  if type(x) == list:
    if len(x) > 0:
      return x[d]
    else:
      return 0
  else:
    return 0

df_reg = df['Стоимость (полная)'].str.findall("\d+")
df['Стоимость от'] = df_reg.apply(get_lists,d=0)
df['Стоимость до'] = df_reg.apply(get_lists,d=1)

In [20]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Стоимость,Стоимость от,Стоимость до
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,бесплатное,фестиваль,культурно-досуговое,NaN,13.06.2020 16:00,NaN,13.06.2020 20:00,NaN,Клуб Современник проведет фестиваль современны...,Нет,NaN,NaN,NaN,недоступно,NaN,0+,от 20 до 40,0,0,0
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,бесплатное,"танцевальный вечер, бал",патриотическое,9 мая. День Победы,09.05.2020 14:30,NaN,09.05.2020 17:30,NaN,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,NaN,16+,от 40 до 85,0,0,0
2,3179,Врасплох,Завершено,ГБУК г. Москвы «ДК «Нагатино»,бесплатное,выставка,антитеррор,NaN,01.01.2020,9:00,31.01.2020,22:00,Регулярное (ежеквартальное) мероприятие цикла ...,Нет,"Судостроительная ул. 31, корпус 1",Южный административный округ,Нагатинский Затон,NaN,NaN,6+,от 16 до 100,0,0,0
3,486,Праздник Новогодней елки. Новогодний концерт в...,Завершено,ГАУК г. Москвы «Культурный центр ЗИЛ»,бесплатное,"концерт, музыкальное представление",культурно-досуговое,1 января. Новый год,01.01.2020,10:00,01.01.2020,19:00,Коллектив Культурного центра ЗИЛ Ансамбль эстр...,Нет,"Россия, Москва",NaN,NaN,NaN,NaN,0+,от 5 до 14,0,0,0
4,2100,"Выставка к Рождеству ""Сувенир Года""",Завершено,ГБУК г.Москвы ТКС «Орехово»,бесплатное,выставка,культурно-досуговое,1 января. Новый год,01.01.2020,10:00,07.01.2020,20:00,В преддверии волшебного праздника Рождества Хр...,Нет,"Борисовский пр. 22, строение 2, корпус 1",Южный административный округ,Орехово-Борисово Северное,недоступно,NaN,6+,от 15 до 80,0,0,0


# Статус

In [21]:
df['Статус'].unique()

array(['Завершено', 'Утверждено', 'Запланировано', 'Отменено',
       'Уточняется', 'Опубликовано'], dtype=object)

In [22]:
df[df['Статус'] == 'Уточняется'].count()

id                                        575
Название мероприятия                      575
Статус                                    575
Наименование учреждения                   575
Стоимость (полная)                        569
Тип мероприятия                           575
Направленность мероприятия                574
Наименование праздника или события        163
Дата начала мероприятия                   575
Время начала мероприятия                  508
Дата окончания мероприятия                575
Время окончания мероприятия               508
Краткое описание                          575
Проводится онлайн                         575
Место проведения                          538
Округ                                     528
Район                                     529
Доступность мероприятия для лиц с ОВЗ     556
Тип ОВЗ                                   159
Возрастной ценз участников мероприятия    575
Возрастная категория                      575
Стоимость                         

In [23]:
df = df[(df['Статус'] != 'Завершено') & (df['Статус'] != 'Отменено')]

In [24]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Стоимость,Стоимость от,Стоимость до
10,1273,Вдохновение Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,недоступно,NaN,0+,от 0 до 90,0,0,0
11,1278,Москва олимпийская,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,недоступно,NaN,0+,от 0 до 90,0,0,0
12,1279,Моё Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,недоступно,NaN,0+,от 0 до 90,0,0,0
13,1280,"Завод, усадьба, ковш и подводная лодка",Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,недоступно,NaN,0+,от 0 до 90,0,0,0
14,1282,По следам старинных усадеб,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,недоступно,NaN,0+,от 0 до 90,0,0,0


In [25]:
df.count()

id                                        5146
Название мероприятия                      5146
Статус                                    5146
Наименование учреждения                   5146
Стоимость (полная)                        4988
Тип мероприятия                           5146
Направленность мероприятия                5028
Наименование праздника или события        1353
Дата начала мероприятия                   5146
Время начала мероприятия                  4433
Дата окончания мероприятия                5146
Время окончания мероприятия               4433
Краткое описание                          5146
Проводится онлайн                         5146
Место проведения                          3702
Округ                                     3580
Район                                     3538
Доступность мероприятия для лиц с ОВЗ     4613
Тип ОВЗ                                   2450
Возрастной ценз участников мероприятия    5146
Возрастная категория                      5146
Стоимость    

# Возрастной ценз

In [26]:
df['Возрастной ценз участников мероприятия'].describe()

count     5146
unique       5
top         6+
freq      1755
Name: Возрастной ценз участников мероприятия, dtype: object

In [27]:
df['Возрастной ценз участников мероприятия'].unique()


array(['0+', '6+', '12+', '18+', '16+'], dtype=object)

In [28]:
df['Возрастной ценз участников мероприятия'].count()

5146

In [29]:
df.loc[df['Возрастной ценз участников мероприятия'] == '0+', 'Возрастной ценз'] = int(0)
df.loc[df['Возрастной ценз участников мероприятия'] == '6+', 'Возрастной ценз'] = int(6)
df.loc[df['Возрастной ценз участников мероприятия'] == '12+', 'Возрастной ценз'] = int(12)
df.loc[df['Возрастной ценз участников мероприятия'] == '16+', 'Возрастной ценз'] = int(16)
df.loc[df['Возрастной ценз участников мероприятия'] == '18+', 'Возрастной ценз'] = int(18)

In [30]:
df = df.drop('Возрастной ценз участников мероприятия', 1)

In [31]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз
10,1273,Вдохновение Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0
11,1278,Москва олимпийская,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,недоступно,NaN,от 0 до 90,0,0,0,0.0
12,1279,Моё Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0
13,1280,"Завод, усадьба, ковш и подводная лодка",Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,недоступно,NaN,от 0 до 90,0,0,0,0.0
14,1282,По следам старинных усадеб,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,недоступно,NaN,от 0 до 90,0,0,0,0.0


In [32]:
df['Возрастной ценз'].unique()

array([ 0.,  6., 12., 18., 16.])

# Проводится онлайн

In [33]:
df.loc[df['Проводится онлайн'] == 'Да', 'Онлайн'] = '1'
df.loc[df['Проводится онлайн'] == 'Нет', 'Онлайн'] = '0'
# df.count()

In [34]:
df['Проводится онлайн'].describe()

count     5146
unique       2
top        Нет
freq      3704
Name: Проводится онлайн, dtype: object

In [35]:
df['Проводится онлайн'].unique()

array(['Нет', 'Да'], dtype=object)

# ОВЗ

In [36]:
df.loc[df['Тип ОВЗ'] != 'NaN', 'ОВЗ'] = df['Тип ОВЗ']
df.loc[df['Доступность мероприятия для лиц с ОВЗ'] == 'недоступно', 'ОВЗ'] = 0
# df.loc[df['Тип ОВЗ'] != 'NaN', 'ОВЗ'] = df['Тип ОВЗ']
# df.count()

In [37]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ
10,1273,Вдохновение Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0
11,1278,Москва олимпийская,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0
12,1279,Моё Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0
13,1280,"Завод, усадьба, ковш и подводная лодка",Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0
14,1282,По следам старинных усадеб,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0


In [38]:
df[df['ОВЗ'] == 0].count()
df['ОВЗ'].fillna(0) 
# df[df['ОВЗ'] == np.NaN].count() 


10                                                       0
11                                                       0
12                                                       0
13                                                       0
14                                                       0
                               ...                        
41069    Опорно-двигательный аппарат, Слух, Инвалиды-ко...
41070                                                    0
41071                                                    0
41072    Опорно-двигательный аппарат, Инвалиды-колясочники
41073                                 Инвалиды-колясочники
Name: ОВЗ, Length: 5146, dtype: object

In [39]:
df['ОВЗ'].unique()

array([0, 'Опорно-двигательный аппарат', 'Слух',
       'Опорно-двигательный аппарат, Слух', nan,
       'Опорно-двигательный аппарат, Инвалиды-колясочники', 'Зрение',
       'Опорно-двигательный аппарат, Слух, Инвалиды-колясочники, Зрение',
       'Опорно-двигательный аппарат, Зрение',
       'Слух, Инвалиды-колясочники', 'Слух, Зрение',
       'Опорно-двигательный аппарат, Слух, Инвалиды-колясочники',
       'Опорно-двигательный аппарат, Слух, Зрение',
       'Опорно-двигательный аппарат, Инвалиды-колясочники, Зрение',
       'Инвалиды-колясочники', 'Слух, Инвалиды-колясочники, Зрение'],
      dtype=object)

In [40]:
def get_lists(x, d=0):
  if type(x) == list:
    if len(x) > 0:
      return 1
    else:
      return 0
  else:
    return 0

df_reg = df['ОВЗ'].str.findall("Опорно-двигательный аппарат")
# print(df_reg)
# df_reg[df_reg == 'Опорно-двигательный аппарат'] = 1
# print(df_reg)
df['Опорно-двигательный аппарат'] = df_reg.apply(get_lists,d=0)
# df['Стоимость до'] = df_reg.apply(get_lists,d=1)

In [41]:
def get_lists(x, d=0):
  if type(x) == list:
    if len(x) > 0:
      return 1
    else:
      return 0
  else:
    return 0

df_reg = df['ОВЗ'].str.findall("Слух")
df['Слух'] = df_reg.apply(get_lists,d=0)

In [42]:
def get_lists(x, d=0):
  if type(x) == list:
    if len(x) > 0:
      return 1
    else:
      return 0
  else:
    return 0

df_reg = df['ОВЗ'].str.findall("Зрение")
df['Зрение'] = df_reg.apply(get_lists,d=0)

In [43]:
def get_lists(x, d=0):
  if type(x) == list:
    if len(x) > 0:
      return 1
    else:
      return 0
  else:
    return 0

df_reg = df['ОВЗ'].str.findall("Инвалиды-колясочники")
df['Инвалиды-колясочники'] = df_reg.apply(get_lists,d=0)

In [44]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
10,1273,Вдохновение Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
11,1278,Москва олимпийская,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
12,1279,Моё Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
13,1280,"Завод, усадьба, ковш и подводная лодка",Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
14,1282,По следам старинных усадеб,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0


# Снова дропаем мусор

In [45]:
df.head()

,id,Название мероприятия,Статус,Наименование учреждения,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
10,1273,Вдохновение Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
11,1278,Москва олимпийская,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
12,1279,Моё Ясенево,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
13,1280,"Завод, усадьба, ковш и подводная лодка",Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
14,1282,По следам старинных усадеб,Утверждено,ГБУК г. Москвы «КЦ «Вдохновение»,бесплатное,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,недоступно,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0


In [46]:
df = df.drop('Стоимость (полная)', 1)
df = df.drop('Статус', 1)
df = df.drop('Доступность мероприятия для лиц с ОВЗ', 1)

# Возраст


In [47]:
df.head()

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
10,1273,Вдохновение Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
11,1278,Москва олимпийская,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
12,1279,Моё Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
13,1280,"Завод, усадьба, ковш и подводная лодка",ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0
14,1282,По следам старинных усадеб,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0


In [48]:
df['Возрастная категория'].describe()

count           5146
unique           354
top       от 6 до 99
freq             226
Name: Возрастная категория, dtype: object

In [49]:
df['Возрастная категория'].unique()

array(['от 0 до 90', 'от 6 до 14', 'от 12 до 99', 'от 35 до 85',
       'от 18 до 35', 'от 18 до 80', 'от 6 до 12', 'от 6 до 90',
       'от 12 до 90', 'от 18 до 45', 'от 35 до 80', 'от 16 до 99',
       'от 0 до 99', 'от 9 до 13', 'от 9 до 14', 'от 12 до 70',
       'от 12 до 16', 'от 16 до 80', 'от 50 до 90', 'от 50 до 95',
       'от 30 до 70', 'от 20 до 70', 'от 30 до 60', 'от 18 до 60',
       'от 6 до 35', 'от 18 до 98', 'от 6 до 70', 'от 18 до 85',
       'от 6 до 80', 'от 18 до 99', 'от 55 до 99', 'от 4', 'от 6 до 16',
       'от 6 до 55', 'от 12 до 30', 'от 8 до 12', 'от 12 до 18',
       'от 4 до 70', 'от 8 до 10', 'от 18 до 75', 'от 4 до 12',
       'от 16 до 70', 'от 16 до 55', 'от 6 до 10', 'от 6 до 99',
       'от 6 до 45', 'от 4 до 10', 'от 16 до 90', 'от 2 до 99',
       'от 6 до 100', 'от 3 до 12', 'от 16 до 60', 'от 12 до 60',
       'от 14 до 80', 'от 14 до 99', 'от 55 до 85', 'от 7 до 14',
       'от 55 до 90', 'от 16 до 25', 'от 16 до 24', 'от 10 до 16',
       'от

In [50]:
df[df['Возрастная категория'] == 'от 4']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
15966,318,Стихи для мамы.,ГБУК г. Москвы «КЦ «Рублево»,"литературные чтения, декламация стихов, литера...",сохранение семейных ценностей,NaN,27.11.2020 17:00,NaN,27.11.2020 18:00,NaN,Стихи для мамы. Праздничное мероприятие ко дню...,Нет,Кунцевская ул. 13/6,Западный административный округ,Кунцево,NaN,от 4,0,0,0,0.0,0,0,0,0,0,0
15968,320,Творческая мастерская «Подарок своими руками»....,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,21.11.2020 12:00,NaN,21.11.2020 13:00,NaN,Для всех любителей бисероплетения будет предос...,Нет,Кунцевская ул. 13/6,Западный административный округ,Кунцево,NaN,от 4,0,0,0,6.0,0,0,0,0,0,0
15971,323,Творческая мастерская «Подарок своими руками»....,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,24.12.2020 19:00,NaN,24.12.2020 20:00,NaN,Творческая мастерская «Подарок своими руками»....,Нет,Кунцевская ул. 13/6,Западный административный округ,Кунцево,NaN,от 4,0,0,0,0.0,0,0,0,0,0,0
16357,709,Сюрпризы Снегурочки,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,1 января. Новый год,26.12.2020 10:00,NaN,26.12.2020 11:00,NaN,Волшебный мастер-класс по изготовлению оригина...,Нет,Василия Ботылева ул. 43,Западный административный округ,Кунцево,NaN,от 4,0,0,0,0.0,0,0,0,0,0,0
16358,710,Сюрпризы Снегурочки,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,1 января. Новый год,26.12.2020 13:00,NaN,26.12.2020 14:00,NaN,Волшебный мастер-класс по изготовлению оригина...,Нет,Василия Ботылева ул. 43,Западный административный округ,Кунцево,NaN,от 4,0,0,0,0.0,0,0,0,0,0,0
16359,711,Сюрпризы Снегурочки,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,1 января. Новый год,27.12.2020 10:00,NaN,27.12.2020 11:00,NaN,Волшебный мастер-класс по изготовлению оригина...,Нет,Василия Ботылева ул. 43,Западный административный округ,Кунцево,NaN,от 4,0,0,0,0.0,0,0,0,0,0,0
16360,712,Сюрпризы Снегурочки,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,1 января. Новый год,27.12.2020 13:00,NaN,27.12.2020 14:00,NaN,Волшебный мастер-класс по изготовлению оригина...,Нет,Василия Ботылева ул. 43,Западный административный округ,Кунцево,NaN,от 4,0,0,0,0.0,0,0,0,0,0,0
26105,10457,У околицы,ГБУК г. Москвы «ДК «Дружба»,"концерт, музыкальное представление",безопасность движения,NaN,31.10.2020 15:00,NaN,31.10.2020 16:00,NaN,Онлайн - концерт Ведущего творческого коллекти...,Да,NaN,NaN,NaN,NaN,от 4,0,0,0,0.0,1,0,0,0,0,0
36760,10456,Воля-вольная,ГБУК г. Москвы «ДК «Дружба»,"концерт, музыкальное представление",культурно-досуговое,1 октября. Международный день пожилых людей,28.10.2020,18:00,28.10.2020,19:00,"В онлайн формате, на платформах социальных се...",Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Зрение",от 4,0,0,0,0.0,1,"Опорно-двигательный аппарат, Зрение",1,0,1,0
37198,10455,Полет,ГБУК г. Москвы «ДК «Стимул»,выставка,культурно-досуговое,NaN,03.11.2020,10:00,17.11.2020,10:00,он-лайн выставка коллектива «Ассорти» на 10 ле...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 4,0,0,0,0.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1


In [51]:
df.loc[df['Возрастная категория'] == 'от 4', 'Возрастная категория'] = 'от 4 до 8'

In [52]:
df[df['Возрастная категория'] == 'от 12']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
23510,7862,Читка Чуковский,ГБУК г. Москвы «ЦК «Сцена»,"литературные чтения, декламация стихов, литера...",культурно-досуговое,NaN,05.12.2020 14:00,NaN,05.12.2020 14:40,NaN,Аудиосказка со счастливым финалом. История одн...,Нет,"Островитянова ул. 15, корпус 1",Юго-Западный административный округ,Коньково,NaN,от 12,0,0,0,12.0,0,NaN,0,0,0,0
35635,10169,Открываем секреты словарей,ГБУК г. Москвы «ЦБС СЗАО»,"комплексное мероприятие, тематическая программа",просветительское,NaN,19.10.2020,18:00,19.10.2020,18:30,Открываем секреты словарей,Да,NaN,NaN,NaN,Опорно-двигательный аппарат,от 12,1,0,0,12.0,1,Опорно-двигательный аппарат,1,0,0,0
35911,11068,"""Мы - за здоровый образ жизни""",ГБУК г.Москвы «ТКС «Спутник»,"интерактивное занятие, мастер-класс, урок",антинаркотическое,NaN,21.10.2020,19:00,21.10.2020,20:00,"В Центре культуры ""Спутник"" состоится спортивн...",Да,NaN,NaN,NaN,NaN,от 12,0,0,0,12.0,1,NaN,0,0,0,0
36500,10171,Мастер-класс Ирины Мартыновой по бумагопластике,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,26.10.2020,15:00,26.10.2020,16:00,На этом мастер-классе ребятам будет предложено...,Да,NaN,NaN,NaN,NaN,от 12,0,0,0,6.0,1,0,0,0,0,0
38019,1218,С Карлсоном и Пеппи веселее жить на свете,ГБУК г. Москвы «ЦБС «Новомосковская»,"комплексное мероприятие, тематическая программа",культурно-досуговое,NaN,14.11.2020,14:00,14.11.2020,16:00,Литературная игра-соревнования по книгам-юбиля...,Нет,"Россия, Москва, Москва, поселок Марьино, д. 3",Новомосковский административный округ,поселение Филимонковское,NaN,от 12,0,0,0,0.0,0,0,0,0,0,0


In [53]:
df.loc[df['Возрастная категория'] == 'от 12', 'Возрастная категория'] = 'от 12 до 14'

In [54]:
df[df['Возрастная категория'] == 'от 6']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
24605,8957,Фантазии синьора Родари,ГБУК г.Москвы «ЦБС ЮЗАО»,викторина,культурно-досуговое,23 октября. 100 лет со дня рождения Джанни Род...,23.10.2020 16:00,NaN,23.10.2020 16:10,NaN,Литературная игра по сказке-повести Джанни Род...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 6,0,0,0,6.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
25219,9571,Юные художники,ГБУК г. Москвы «ТКС «Фили-Давыдково»,выставка,культурно-досуговое,NaN,19.10.2020 11:00,NaN,23.10.2020 19:00,NaN,"С 19 по 23 октября в социальной сети ""ВКонтакт...",Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Слух, Инвалиды-ко...",от 6,0,0,0,0.0,1,"Опорно-двигательный аппарат, Слух, Инвалиды-ко...",1,1,0,1
25349,9701,Угадай мелодию!,ГБУК г. Москвы «ТКС «Товарищ»,викторина,культурно-досуговое,NaN,21.10.2020 16:00,NaN,21.10.2020 16:45,NaN,Музыкальная викторина для школьников,Да,Алтуфьевское шоссе 22,Северо-Восточный административный округ,Отрадное,NaN,от 6,0,0,0,6.0,1,0,0,0,0,0
26321,10673,Танцуем классику,ГБУК г. Москвы «ТКС «Товарищ»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,18.11.2020 19:30,NaN,18.11.2020 20:15,NaN,Мастер-класс по классической хореографии,Нет,Алтуфьевское шоссе 22,Северо-Восточный административный округ,Отрадное,NaN,от 6,0,0,0,6.0,0,0,0,0,0,0
27150,11502,Цветы для самой любимой!,ГБУК г. Москвы «ТКС «Товарищ»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,25 ноября. День матери,23.11.2020 18:00,NaN,23.11.2020 18:40,NaN,Мастер- класс по прикладному творчеству,Нет,Алтуфьевское шоссе 22,Северо-Восточный административный округ,Отрадное,NaN,от 6,0,0,0,6.0,0,0,0,0,0,0
34731,7869,"Выставка ""Мудрость мастеров""",ГБУК г. Москвы «КЦ «Авангард»,выставка,культурно-досуговое,NaN,10.10.2020,9:00,31.10.2020,22:00,Авторская выставка руководителя студии восточн...,Да,NaN,NaN,NaN,NaN,от 6,0,0,0,6.0,1,0,0,0,0,0
35637,9221,Осенние деньки,ГБУК г. Москвы «ТКС «Товарищ»,"встреча, творческий вечер",культурно-досуговое,NaN,19.10.2020,18:00,19.10.2020,19:00,Музыкальный вечер Софии Махмудовой,Да,"Мира просп. 163, корпус 1",Северо-Восточный административный округ,Ростокино,NaN,от 6,0,0,0,6.0,1,0,0,0,0,0
35786,10179,"Мастер-класс ""Забавные животные""",ГБУК г. Москвы «ЦБС СЗАО»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,20.10.2020,20:00,20.10.2020,20:30,"Мастер-класс ""Забавные животные""",Да,NaN,NaN,NaN,Опорно-двигательный аппарат,от 6,1,0,0,6.0,1,Опорно-двигательный аппарат,1,0,0,0
35870,8958,Лягушка-путешественница,ГБУК г.Москвы «ЦБС ЮЗАО»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,21.10.2020,16:00,21.10.2020,16:10,В социальных сетях библиотеки №170 для юных чи...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 6,0,0,0,6.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
36037,9510,Рисуем стихи: Закаляка,ГБУК г.Москвы «ЦБС ЮЗАО»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,22.10.2020,16:00,22.10.2020,16:15,Мастер-класс по иллюстрации стихотворения на к...,Да,NaN,NaN,NaN,NaN,от 6,0,0,0,0.0,1,NaN,0,0,0,0


In [55]:
df.loc[df['Возрастная категория'] == 'от 6', 'Возрастная категория'] = 'от 6 до 18'

In [56]:
df[df['Возрастная категория'] == 'от 5']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
25267,9619,Фуэте,ГБУК г. Москвы «ТКС «Товарищ»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,28.10.2020 17:00,NaN,28.10.2020 17:45,NaN,Класс-концерт по хореографии,Да,"Лескова ул. 30, корпус 1",Северо-Восточный административный округ,Бибирево,NaN,от 5,0,0,0,0.0,1,0,0,0,0,0
25270,9622,Что почитать: жуткие книги на Хеллоуин,ГБУК г.Москвы «ЦБС ЮЗАО»,"презентация, демонстрация, показетльные выступ...",культурно-досуговое,NaN,27.10.2020 17:00,NaN,27.10.2020 17:20,NaN,Цикл рекомендательных видео. Подборка книг на ...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 5,0,0,0,12.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
35562,9625,Леди осень,ГБУК г. Москвы «ДК «Стимул»,выставка,культурно-досуговое,NaN,19.10.2020,10:00,31.10.2020,10:00,Танцевальный коллектив «Виктория» подготовил ф...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 5,0,0,0,18.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
35789,9621,"""Арт-завтрак"" в Родительском клубе ""Моя семья""...",ГБУК г. Москвы «ТКС «Ново-Переделкино»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,21.10.2020,10:00,21.10.2020,11:00,"""Арт-завтрак"" в Родительском клубе ""Моя семья""...",Нет,Шолохова ул. 12,Западный административный округ,Ново-Переделкино,NaN,от 5,0,0,0,0.0,0,0,0,0,0,0
36506,9624,Па-де-де,ГБУК г. Москвы «ТКС «Товарищ»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,26.10.2020,16:30,26.10.2020,17:00,Класс-концерт по хореографии,Да,"Лескова ул. 30, корпус 1",Северо-Восточный административный округ,Бибирево,NaN,от 5,0,0,0,0.0,1,0,0,0,0,0
37509,1650,Мы вместе!,ГБУК города Москвы «Дом культуры «Восход»,"встреча, творческий вечер",просветительское,NaN,05.11.2020,17:00,05.11.2020,18:00,День народного единства (4 ноября),Нет,NaN,NaN,NaN,NaN,от 5,0,0,0,0.0,0,0,0,0,0,0
37559,1668,Единство - сила,ГБУК г.Москвы «ЦБС ЮВАО»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,4 ноября. День народного единства,06.11.2020,15:00,06.11.2020,17:00,В честь акции волонтеры библиотеки объединятся...,Нет,Перерва ул. 56/2,Юго-Восточный административный округ,Марьино,NaN,от 5,0,0,0,6.0,0,0,0,0,0,0
38826,9615,"Огонь - друг, огонь - враг",ГБУК г. Москвы «ТКС «Товарищ»,"дискуссия, обсуждение, заседание",просветительское,NaN,25.11.2020,17:00,25.11.2020,17:30,Информационное занятие направлено на закреплен...,Нет,"Лескова ул. 30, корпус 1",Северо-Восточный административный округ,Бибирево,NaN,от 5,0,0,0,0.0,0,0,0,0,0,0
40021,1672,РосКвиз,ГБУК г.Москвы «ЦБС ЮВАО»,"комплексное мероприятие, тематическая программа",культурно-досуговое,12 декабря. День Конституции Российской Федерации,13.12.2020,13:00,13.12.2020,14:00,Приглашаем гостей и читателей Библиотеки 129 и...,Нет,Перерва ул. 56/2,Юго-Восточный административный округ,Марьино,NaN,от 5,0,0,0,12.0,0,0,0,0,0,0
40469,1649,"""Новый год к нам мчится""","ГБУК г. Москвы ""ДК ""Темп""",акция,культурно-досуговое,1 января. Новый год,21.12.2020,17:00,31.12.2020,21:00,Выездные поздравления Деда Мороза и Снегурочки,Нет,NaN,NaN,NaN,NaN,от 5,1,1500,1500,6.0,0,NaN,0,0,0,0


In [57]:
df.loc[df['Возрастная категория'] == 'от 5', 'Возрастная категория'] = 'от 5 до 12'

In [58]:
df[df['Возрастная категория'] == 'от 8']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
26141,10493,"Флешмоб ""Лидер - за здоровый образ жизни!""",ГБУК г. Москвы «КЦ «Лидер»,акция,антинаркотическое,NaN,30.10.2020 09:00,NaN,30.10.2020 22:00,NaN,30 октября во всех социальных сетях культурног...,Да,NaN,NaN,NaN,Опорно-двигательный аппарат,от 8,0,0,0,0.0,1,Опорно-двигательный аппарат,1,0,0,0
35235,10492,Осенние листочки,ГБУК г. Москвы «ТКС «Ново-Переделкино»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,15.10.2020,16:00,15.11.2020,17:00,"Мастер-класс по вышивке ""Осенние листочки"" Вед...",Да,NaN,NaN,NaN,NaN,от 8,0,0,0,6.0,1,NaN,0,0,0,0
37377,11663,Ночь искусств: простофокус,ГБУК г. Москвы «ДК «Берендей»,"комплексное мероприятие, тематическая программа",культурно-досуговое,3 ноября. 125 лет со дня рождения Э.Г.Багрицкого,03.11.2020,19:00,03.11.2020,21:00,"Мероприятие в онлайн-формате, посвященное вось...",Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 8,0,0,0,6.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
37884,11664,Показательное выступление студии карате. Трене...,"ГБУК г. Москвы ""ДК ""Темп""","комплексное мероприятие, тематическая программа","спортивное, пропаганда здорового образа жизни",NaN,12.11.2020,15:00,19.11.2020,16:00,Показательное выступление студии карате. Трене...,Да,NaN,NaN,NaN,NaN,от 8,1,0,0,6.0,1,NaN,0,0,0,0
38168,1024,"Дом, милый дом",ГБУК г. Москвы «ТКС «Ново-Переделкино»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,17.11.2020,15:00,17.11.2020,16:00,"Мастер-класс ""Дом, милый дом"" по созданию текс...",Нет,"Чоботовская ул. 17, корпус 1",Западный административный округ,Ново-Переделкино,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 8,0,0,0,6.0,0,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
38335,1060,"Аккорд, ещё один аккорд",ГБУК г. Москвы «ТКС «Ново-Переделкино»,"комплексное мероприятие, тематическая программа",культурно-досуговое,NaN,19.11.2020,18:00,19.11.2020,19:00,"Открытый урок ""Аккорд, ещё один аккорд"" в круж...",Нет,Боровское шоссе 40,Западный административный округ,Ново-Переделкино,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 8,0,0,0,6.0,0,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1


In [59]:
df.loc[df['Возрастная категория'] == 'от 8', 'Возрастная категория'] = 'от 8 до 12'

In [60]:
df[df['Возрастная категория'] == 'от 35']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
35586,9598,Про всё на свете,ГБУК г.Москвы «ЦБС ЮЗАО»,викторина,культурно-досуговое,NaN,19.10.2020,12:00,20.10.2020,13:00,Знакомство читателей библиотеки с творчеством ...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 35,0,0,0,0.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1


In [61]:
df.loc[df['Возрастная категория'] == 'от 35', 'Возрастная категория'] = 'от 35 до 90'

In [62]:
df[df['Возрастная категория'] == 'от 3']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
35758,11596,Парад премьер,ГБУК г. Москвы «ТКС «Ново-Переделкино»,"концерт, музыкальное представление",культурно-досуговое,NaN,20.10.2020,17:00,20.11.2020,18:00,"Концерт ""Парад премьер"" - смотр хореографическ...",Да,NaN,NaN,NaN,NaN,от 3,0,0,0,0.0,1,NaN,0,0,0,0
37389,11597,Лоскутное одеяло,ГБУК г. Москвы «ДК «Дружба»,"концерт, музыкальное представление",культурно-досуговое,NaN,03.11.2020,20:00,03.11.2020,21:00,В прямом эфире на платформе Instagramm пройдёт...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 3,0,0,0,6.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
37637,11595,23 несчастья,ГБУК г. Москвы «ЦК «Сцена»,"представление, спектакль, инсценировка, перфом...",культурно-досуговое,NaN,07.11.2020,18:00,07.11.2020,19:00,"""23 несчастья"" - это романтическая комедия сит...",Нет,"Островитянова ул. 15, корпус 1",Юго-Западный административный округ,Коньково,NaN,от 3,1,500,700,0.0,0,NaN,0,0,0,0
38640,11449,"""Мама-главное слово!""",ГБУК г. Москвы «ЦК «Сцена»,"комплексное мероприятие, тематическая программа",культурно-досуговое,25 ноября. День матери,22.11.2020,12:00,22.11.2020,14:30,"Центр Культуры ""Сцена"" приглашает на семейную ...",Нет,"Островитянова ул. 15, корпус 1",Юго-Западный административный округ,Коньково,"Опорно-двигательный аппарат, Слух, Инвалиды-ко...",от 3,0,0,0,0.0,0,"Опорно-двигательный аппарат, Слух, Инвалиды-ко...",1,1,1,1
39790,321,Клуб «Литературная фантазия».,ГБУК г. Москвы «КЦ «Рублево»,"литературные чтения, декламация стихов, литера...",просветительское,NaN,08.12.2020,19:00,08.12.2020,20:20,Клуб «Литературная фантазия». Путешествие в ми...,Нет,Кунцевская ул. 13/6,Западный административный округ,Кунцево,NaN,от 3,0,0,0,0.0,0,0,0,0,0,0
40119,322,Аппликация из бумаги. «Друзья на льдине».,ГБУК г. Москвы «КЦ «Рублево»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,15.12.2020,19:00,15.12.2020,20:00,Аппликация из цветной бумаги на тему «Животные...,Нет,Кунцевская ул. 13/6,Западный административный округ,Кунцево,NaN,от 3,0,0,0,0.0,0,0,0,0,0,0


In [63]:
df.loc[df['Возрастная категория'] == 'от 3', 'Возрастная категория'] = 'от 3 до 5'

In [64]:
df[df['Возрастная категория'] == 'от 16']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
36835,9573,Закладка-собачка,ГБУК г.Москвы «ЦБС ЮЗАО»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,29.10.2020,15:00,29.10.2020,15:30,29 октября в соц. сетях библиотеки № 196 будет...,Да,NaN,NaN,NaN,Инвалиды-колясочники,от 16,0,0,0,6.0,1,Инвалиды-колясочники,0,0,0,1
37979,11454,"""Ретро""",ГБУК г. Москвы «ЦК «Сцена»,"представление, спектакль, инсценировка, перфом...",культурно-досуговое,NaN,13.11.2020,19:00,13.11.2020,20:30,"Центр Культуры ""Сцена"" приглашает гостей на сп...",Нет,"Островитянова ул. 15, корпус 1",Юго-Западный административный округ,Коньково,NaN,от 16,0,0,0,16.0,0,NaN,0,0,0,0
38219,11455,В мир права через мир сказок,ГБУК г.Москвы «ЦБС ЮЗАО»,"литературные чтения, декламация стихов, литера...",просветительское,NaN,18.11.2020,14:00,18.11.2020,14:30,Литературная программа для детей,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 16,0,0,0,0.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1
38567,9572,Ночь PRO свет,ГБУК г. Москвы «КЦ «Зодчие»,акция,культурно-досуговое,NaN,20.11.2020,19:00,20.11.2020,23:00,Культурно-просветительская акция ’’Ночь PRO св...,Нет,Партизанская ул. 23,Западный административный округ,Кунцево,Опорно-двигательный аппарат,от 16,0,0,0,16.0,0,Опорно-двигательный аппарат,1,0,0,0


In [65]:
df.loc[df['Возрастная категория'] == 'от 16', 'Возрастная категория'] = 'от 12 до 16'

In [66]:
df[df['Возрастная категория'] == 'от 14']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
36876,8874,Голос как инструмент,ГБУК г. Москвы «ДК «Кленово»,"интерактивное занятие, мастер-класс, урок",просветительское,NaN,29.10.2020,18:00,29.10.2020,19:00,Открытый мастер-класс в вокальном коллективе ...,Да,NaN,NaN,NaN,"Опорно-двигательный аппарат, Инвалиды-колясочники",от 14,0,0,0,6.0,1,"Опорно-двигательный аппарат, Инвалиды-колясочники",1,0,0,1


In [67]:
df.loc[df['Возрастная категория'] == 'от 14', 'Возрастная категория'] = 'от 10 до 16'

In [68]:
df[df['Возрастная категория'] == 'от 50']

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники
37117,11731,Лира,ГБУК г. Москвы «ЦКИ «Меридиан»,"литературные чтения, декламация стихов, литера...",просветительское,3 октября. 125 лет со дня рождения С.А.Есенина,01.11.2020,16:00,01.11.2020,17:00,"Музыкально- поэтическая программа, посвященная...",Да,NaN,NaN,NaN,NaN,от 50,0,0,0,12.0,1,0,0,0,0,0
38364,11732,Букет для любимой Мамы,ГБУК г. Москвы «ДК «Стимул»,выставка,культурно-досуговое,25 ноября. День матери,20.11.2020,10:00,06.12.2020,10:00,Выставка «Букет для любимой Мамы» Ведущей Твор...,Нет,"Сибирский пр. 2, строение 5",Центральный административный округ,Таганский,NaN,от 50,0,0,0,6.0,0,NaN,0,0,0,0
38920,9950,Осенний салат,ГБУК г. Москвы «ЦКИ «Меридиан»,"интерактивное занятие, мастер-класс, урок",просветительское,NaN,26.11.2020,15:00,26.11.2020,16:30,"Мастер- класс в клубе домоводства ""Хозяюшка"" д...",Да,"Профсоюзная ул. 61, Конференц-зал (231 каб.)",Юго-Западный административный округ,Черёмушки,NaN,от 50,0,0,0,12.0,1,0,0,0,0,0
39037,9956,Мастер- класс по историко- бытовому и бальному...,ГБУК г. Москвы «ЦКИ «Меридиан»,"интерактивное занятие, мастер-класс, урок",культурно-досуговое,NaN,27.11.2020,16:00,27.11.2020,17:30,Культурный центр «Меридиан» с особой заботой о...,Да,"Профсоюзная ул. 61, Фойе 2-го этажа (с кулуарами)",Юго-Западный административный округ,Черёмушки,NaN,от 50,0,0,0,12.0,1,0,0,0,0,0
39217,9961,Осенний бал,ГБУК г. Москвы «ЦКИ «Меридиан»,"танцевальный вечер, бал",культурно-досуговое,1 октября. Международный день пожилых людей,29.11.2020,15:00,29.11.2020,17:00,В ретро-программе – выступление постоянных уча...,Нет,"Профсоюзная ул. 61, Фойе 2-го этажа (с кулуарами)",Юго-Западный административный округ,Черёмушки,NaN,от 50,0,0,0,12.0,0,0,0,0,0,0
39712,5994,Спектакль современного танца Libertatem,ГАУК г. Москвы «Культурный центр ЗИЛ»,"представление, спектакль, инсценировка, перфом...",культурно-досуговое,NaN,06.12.2020,17:00,06.12.2020,19:00,Своей первый спектакль компания современного т...,Нет,"Восточная ул. 4, корпус 1",Южный административный округ,Даниловский,NaN,от 50,1,650,650,12.0,0,0,0,0,0,0


In [69]:
df.loc[df['Возрастная категория'] == 'от 50', 'Возрастная категория'] = 'от 50 до 90'

In [70]:
def get_lists(x, d=0):
  if type(x) == list:
    if len(x) > 0:
      # print(len(x), x)
      try:
        return int(x[d])
      except:
        return int(90)
    else:
      return 0
  else:
    return 0

df_reg = df['Возрастная категория'].str.findall("\d+")
df['Возраст от'] = df_reg.apply(get_lists,d=0)
df['Возраст до'] = df_reg.apply(get_lists,d=1)

In [71]:
df.head()

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Возрастная категория,Стоимость,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники,Возраст от,Возраст до
10,1273,Вдохновение Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0,0,90
11,1278,Москва олимпийская,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0,0,90
12,1279,Моё Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0,0,90
13,1280,"Завод, усадьба, ковш и подводная лодка",ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0,0,90
14,1282,По следам старинных усадеб,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,NaN,от 0 до 90,0,0,0,0.0,0,0,0,0,0,0,0,90


In [72]:
df = df.drop('Стоимость', 1)
df = df.drop('Возрастная категория', 1)
# df = df.drop('Доступность мероприятия для лиц с ОВЗ', 1)

In [73]:
df.head()

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники,Возраст от,Возраст до
10,1273,Вдохновение Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,NaN,0,0,0.0,0,0,0,0,0,0,0,90
11,1278,Москва олимпийская,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,NaN,0,0,0.0,0,0,0,0,0,0,0,90
12,1279,Моё Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,NaN,0,0,0.0,0,0,0,0,0,0,0,90
13,1280,"Завод, усадьба, ковш и подводная лодка",ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,NaN,0,0,0.0,0,0,0,0,0,0,0,90
14,1282,По следам старинных усадеб,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,NaN,0,0,0.0,0,0,0,0,0,0,0,90


In [74]:
max = 0
for row in df['Дата начала мероприятия']:
  # print(row)
  try:
    if len(row) > max:
      max = len(row)
  except:
    pass
print(max)

16


In [75]:
df.to_csv('/content/drive/My Drive/Datasets_LCT/new_events.csv')

# Облако пользователя

In [76]:
# from datetime import date
from datetime import datetime

def calculate_age(born):
  today = datetime.today()
  return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [77]:
class User:
  def __init__(self):
    self.date = '1991-10-01'
    # print(calculateAge(date(int(self.date[0:3]), int(self.date[5:6]), int(self.date[8:9]))), "years") 
    # self.age = calculate_age(date(int(self.date[0:3]), int(self.date[5:6]), int(self.date[8:9])))
    date = datetime.strptime(self.date, '%Y-%m-%d')
    self.age = calculate_age(date)
    self.InterestCloud = ['история', 'астрономия', 'чехов', 'толстой', 'джаз', 'уют', 'ярмарки', 'классическая', 'орган', 'импрессионизм']
    self.Correlation = [0] * (len(self.InterestCloud) + 1)
    self.oda = 1
    self.hearing = 0
    self.vision = 0
    self.wheelchair = 0


In [78]:
User = User()
# print(User.age)

In [79]:
df.head()

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники,Возраст от,Возраст до
10,1273,Вдохновение Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста обойдут весь жилой район Ясе...,Нет,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",Юго-Западный административный округ,Ясенево,NaN,0,0,0.0,0,0,0,0,0,0,0,90
11,1278,Москва олимпийская,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,"Участники квеста посетят места, неразрывно свя...",Нет,"Россия, Москва, Москва, ст. м. Проспект Вернад...",Западный административный округ,Тропарёво-Никулино,NaN,0,0,0.0,0,0,0,0,0,0,0,90
12,1279,Моё Ясенево,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,18 мая. Международный день музеев,01.01.2020,10:00,31.12.2020,22:00,Участники получат возможность увидеть весь жи...,Нет,"Россия, Москва, Москва, ст. м. Ясенево",Юго-Западный административный округ,Ясенево,NaN,0,0,0.0,0,0,0,0,0,0,0,90
13,1280,"Завод, усадьба, ковш и подводная лодка",ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Участники квеста посетят московский район Туши...,Нет,"Россия, Москва, Москва, ст. м. Тушино",Юго-Западный административный округ,Северное Тушино,NaN,0,0,0.0,0,0,0,0,0,0,0,90
14,1282,По следам старинных усадеб,ГБУК г. Москвы «КЦ «Вдохновение»,квест,культурно-досуговое,NaN,01.01.2020,10:00,31.12.2020,22:00,Квест знакомит с рядом исторических усадеб юг...,Нет,"Россия, Москва, Москва, Загородное шоссе, вл. 2",Южный административный округ,Донской,NaN,0,0,0.0,0,0,0,0,0,0,0,90


In [80]:
df.count()
data = df.copy()

# df = data.copy()

In [81]:
print(df.count())
df = df[df['Возрастной ценз'] <= User.age]
print(df.count())

id                                    5146
Название мероприятия                  5146
Наименование учреждения               5146
Тип мероприятия                       5146
Направленность мероприятия            5028
Наименование праздника или события    1353
Дата начала мероприятия               5146
Время начала мероприятия              4433
Дата окончания мероприятия            5146
Время окончания мероприятия           4433
Краткое описание                      5146
Проводится онлайн                     5146
Место проведения                      3702
Округ                                 3580
Район                                 3538
Тип ОВЗ                               2450
Стоимость от                          5146
Стоимость до                          5146
Возрастной ценз                       5146
Онлайн                                5146
ОВЗ                                   4620
Опорно-двигательный аппарат           5146
Слух                                  5146
Зрение     

In [82]:

df = df[df['Опорно-двигательный аппарат'] >= User.oda]
print(df.count())
df = df[df['Слух'] >= User.hearing]
print(df.count())
df = df[df['Зрение'] >= User.vision]
print(df.count())
df = df[df['Инвалиды-колясочники'] >= User.wheelchair]
print(df.count())

id                                    2267
Название мероприятия                  2267
Наименование учреждения               2267
Тип мероприятия                       2267
Направленность мероприятия            2212
Наименование праздника или события     672
Дата начала мероприятия               2267
Время начала мероприятия              1941
Дата окончания мероприятия            2267
Время окончания мероприятия           1941
Краткое описание                      2267
Проводится онлайн                     2267
Место проведения                      1502
Округ                                 1458
Район                                 1444
Тип ОВЗ                               2267
Стоимость от                          2267
Стоимость до                          2267
Возрастной ценз                       2267
Онлайн                                2267
ОВЗ                                   2267
Опорно-двигательный аппарат           2267
Слух                                  2267
Зрение     

In [83]:
df.head()

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники,Возраст от,Возраст до
456,472,Успешные профессии,ГБУК г. Москвы «ЦБС ВАО»,"комплексное мероприятие, тематическая программа",просветительское,NaN,01.08.2020 12:00,NaN,30.11.2020 18:00,NaN,Встречи с профессионалами из сферы креативных ...,Нет,NaN,NaN,NaN,Опорно-двигательный аппарат,0,0,6.0,0,Опорно-двигательный аппарат,1,0,0,0,6,14
717,6,"Лекции из цикла ""Образовательная среда""",ГБУК г. Москвы «Библиотека-читальня им. И.С. Т...,лекция,культурно-досуговое,NaN,09.01.2020,15:00,23.12.2020,17:00,"Проект ""Образовательная среда"" стартовал в апр...",Нет,"Бобров пер. 6, строение 2, Тургеневская гостин...",Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99
718,63,"Лекции из цикла ""Образовательная среда""",ГБУК г. Москвы «Библиотека-читальня им. И.С. Т...,лекция,культурно-досуговое,NaN,09.01.2020,15:00,23.12.2020,17:00,"Проект ""Образовательная среда"" стартовал в апр...",Нет,"Бобров пер. 6, строение 2, Тургеневская гостин...",Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99
720,1739,"Лекции из цикла ""Образовательная среда""",ГБУК г. Москвы «Библиотека-читальня им. И.С. Т...,лекция,культурно-досуговое,NaN,09.01.2020,15:00,23.12.2020,17:00,"Проект ""Образовательная среда"" стартовал в апр...",Нет,"Бобров пер. 6, строение 2, Тургеневская гостин...",Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99
2839,401,Вечера Русского ПЕНа. Творческие встречи с пис...,ГБУК г. Москвы «Библиотека искусств им. А.П. Б...,"встреча, творческий вечер",просветительское,NaN,29.01.2020,19:00,24.12.2020,21:00,"Творческие вечера поэтов, прозаиков, драматург...",Нет,"Сущевская ул. 14, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,18.0,0,Опорно-двигательный аппарат,1,0,0,0,18,80


In [84]:
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS

In [85]:
def sim_func(InterestsWordDict, answer):
  for user_word in InterestsWordDict:
    for word in answer:
      if word == user_word:
        return 1
  return 0

In [86]:
def find_dict(user_word):
  word = morph.parse(user_word)[0]
  lexemes = word.lexeme
  answer = [word.word for word in lexemes]
  word_dict = list(set(answer))
  return word_dict

In [88]:
from tqdm import tqdm

indexes = []
types = []
all_user_words = []

for index, row in tqdm(df.iterrows()):
  # print(row['Возраст от'])
  User.Correlation[0] = 1 if (int(row['Возраст от']) <= User.age) and (int(row['Возраст до']) >= User.age) else 0
  # print(User.Correlation[0])
  sim_sum = 0
  for key in User.InterestCloud:
    keyword = key.lower()
    str_orig = row['Краткое описание']
    str_low = str_orig.lower()
    # string = str_low.strip('"', "'", "*", "&", ",", ".", "?", "!")
    # string = re.sub(r'\W', '', str_low)
    str_pre = re.sub(r'[^а-яА-Я ]+', '', str_low)
    string = str_pre.strip()

    

    words = string.split()
    functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
    answer = [word for word in words if pos(word) not in functors_pos]
    # print(answer)

    InterestsWordDict = find_dict(keyword)    
    sim = sim_func(InterestsWordDict, answer) 
    sim_sum += sim
    flag = False
    for us_word in all_user_words:
      if sim_sum and us_word != keyword:
        flag = True
        all_user_words.append(keyword)
    if flag:
      bonus += 0.1

  # if sim_sum > 0:
    # print(sim_sum)
  corr = User.Correlation[0] + (sim_sum/len(User.InterestCloud))
  if corr > 1.05:
    indexes.append(index)
    bonus = 0
    flag = True
    for typ in types: 
      if row['Тип мероприятия'].lower() == typ.lower():
        flag = False
    if flag:
      bonus += 0.1
    types.append(row['Тип мероприятия'])

    df.loc[[index], 'Score'] = corr + bonus

    # print(answer)
    # replaced_string = a_string.replace('"', "")

2267it [03:14, 11.69it/s]


In [89]:
df.loc[indexes, : ]
# df[df['id'] == 36270]

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники,Возраст от,Возраст до,Score
717,6,"Лекции из цикла ""Образовательная среда""",ГБУК г. Москвы «Библиотека-читальня им. И.С. Т...,лекция,культурно-досуговое,NaN,09.01.2020,15:00,23.12.2020,17:00,"Проект ""Образовательная среда"" стартовал в апр...",Нет,"Бобров пер. 6, строение 2, Тургеневская гостин...",Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99,1.2
718,63,"Лекции из цикла ""Образовательная среда""",ГБУК г. Москвы «Библиотека-читальня им. И.С. Т...,лекция,культурно-досуговое,NaN,09.01.2020,15:00,23.12.2020,17:00,"Проект ""Образовательная среда"" стартовал в апр...",Нет,"Бобров пер. 6, строение 2, Тургеневская гостин...",Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99,1.1
720,1739,"Лекции из цикла ""Образовательная среда""",ГБУК г. Москвы «Библиотека-читальня им. И.С. Т...,лекция,культурно-досуговое,NaN,09.01.2020,15:00,23.12.2020,17:00,"Проект ""Образовательная среда"" стартовал в апр...",Нет,"Бобров пер. 6, строение 2, Тургеневская гостин...",Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99,1.1
2839,401,Вечера Русского ПЕНа. Творческие встречи с пис...,ГБУК г. Москвы «Библиотека искусств им. А.П. Б...,"встреча, творческий вечер",просветительское,NaN,29.01.2020,19:00,24.12.2020,21:00,"Творческие вечера поэтов, прозаиков, драматург...",Нет,"Сущевская ул. 14, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,18.0,0,Опорно-двигательный аппарат,1,0,0,0,18,80,1.2
2840,952,Вечера Русского ПЕНа. Творческие встречи с пис...,ГБУК г. Москвы «Библиотека искусств им. А.П. Б...,"встреча, творческий вечер",просветительское,NaN,29.01.2020,19:00,24.12.2020,21:00,"Творческие вечера поэтов, прозаиков, драматург...",Нет,"Сущевская ул. 14, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,18.0,0,Опорно-двигательный аппарат,1,0,0,0,18,80,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40653,2564,Зима рисует снежные узоры,ГБУК г. Москвы «ЦБС СЗАО»,"комплексное мероприятие, тематическая программа",культурно-досуговое,1 января. Новый год,24.12.2020,14:00,24.12.2020,15:30,В программе мастер-классы по изготовлению ново...,Нет,"Мневники ул. 10, корпус 1",Северо-Западный административный округ,Хорошёво-Мнёвники,Опорно-двигательный аппарат,0,0,6.0,0,Опорно-двигательный аппарат,1,0,0,0,6,90,1.1
40800,756,Новогодний рейс,ГБУК г. Москвы «ЦБС ЦАО»,"просмотр фильмов, мультфильмов, кинолекторий, ...",культурно-досуговое,1 января. Новый год,25.12.2020,18:30,25.12.2020,22:00,"""В преддверии волшебного праздника детства Нов...",Нет,"Ружейный пер. 4, строение 1",Центральный административный округ,Хамовники,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99,1.1
40805,8414,Зимняя сказка,ГБУК г. Москвы «ЦБС ЦАО»,"концерт, музыкальное представление",культурно-досуговое,1 января. Новый год,25.12.2020,19:00,25.12.2020,20:00,Концертная программа с участием молодых исполн...,Нет,"Банный пер. 4, строение 2",Центральный административный округ,Мещанский,Опорно-двигательный аппарат,0,0,16.0,0,Опорно-двигательный аппарат,1,0,0,0,16,70,1.1
40956,626,В гостях у сказки,ГБУК г. Москвы «ДК «Дружба»,"концерт, музыкальное представление",культурно-досуговое,1 января. Новый год,27.12.2020,17:00,27.12.2020,18:00,"Хореографический коллектив ""ЭСТЕМ"" поздрави

In [90]:
df['Score'].unique()

array([nan, 1.2, 1.1, 1.2])

In [91]:
# df[df['Score'] >= 1.2]

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники,Возраст от,Возраст до,Score
717,6,"Лекции из цикла ""Образовательная среда""",ГБУК г. Москвы «Библиотека-читальня им. И.С. Т...,лекция,культурно-досуговое,NaN,09.01.2020,15:00,23.12.2020,17:00,"Проект ""Образовательная среда"" стартовал в апр...",Нет,"Бобров пер. 6, строение 2, Тургеневская гостин...",Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,99,1.2
2839,401,Вечера Русского ПЕНа. Творческие встречи с пис...,ГБУК г. Москвы «Библиотека искусств им. А.П. Б...,"встреча, творческий вечер",просветительское,NaN,29.01.2020,19:00,24.12.2020,21:00,"Творческие вечера поэтов, прозаиков, драматург...",Нет,"Сущевская ул. 14, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,18.0,0,Опорно-двигательный аппарат,1,0,0,0,18,80,1.2
15666,18,Я выбираю жизнь,ГБУК г. Москвы «ЦБС ЦАО»,"интерактивное занятие, мастер-класс, урок",антинаркотическое,NaN,01.12.2020 14:00,NaN,01.12.2020 15:00,NaN,01 декабря 2020 года библиотека №267 имени Н. ...,Нет,Спасская Б. ул. 31,Центральный административный округ,Красносельский,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,70,1.2
16131,483,Унесенные ветром,ГБУК г. Москвы «ЦБС ЦАО»,"просмотр фильмов, мультфильмов, кинолекторий, ...",культурно-досуговое,NaN,08.11.2020 10:00,NaN,08.11.2020 12:00,NaN,В 1936 году увидел свет роман американской пис...,Нет,"Чистопрудный бульв. 23, строение 1",Центральный административный округ,Басманный,Опорно-двигательный аппарат,0,0,18.0,0,Опорно-двигательный аппарат,1,0,0,0,18,75,1.2
16463,815,"Толстые журналы. Прошлое, настоящее, будущее",ГБУК г. Москвы «ЦБС ЦАО»,"конференция, круглый стол",просветительское,NaN,29.10.2020 19:00,NaN,29.10.2020 21:00,NaN,Чеховский культурный центр приглашает на кругл...,Нет,"Страстной бульв. 6, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,16.0,0,Опорно-двигательный аппарат,1,0,0,0,16,60,1.2
16465,817,Сувенир,ГБУК г. Москвы «ЦБС ЦАО»,"дискуссия, обсуждение, заседание",просветительское,NaN,11.11.2020 19:00,NaN,11.11.2020 21:00,NaN,"""Чеховский культурный центр в рамках проекта ...",Нет,"Страстной бульв. 6, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,16.0,0,Опорно-двигательный аппарат,1,0,0,0,16,60,1.2
16467,819,Чехов и остров Сахалин,ГБУК г. Москвы «ЦБС ЦАО»,"презентация, демонстрация, показетльные выступ...",культурно-досуговое,NaN,17.11.2020 19:00,NaN,17.11.2020 22:00,NaN,"""Чеховский культурный центр приглашает га откр...",Нет,"Страстной бульв. 6, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,12.0,0,Опорно-двигательный аппарат,1,0,0,0,12,60,1.2
18216,2568,Тепло Нового года,ГБУК г. Москвы «ЦБС СЗАО»,"комплексное мероприятие, тематическая программа",сохранение семейных ценностей,1 января. Новый год,25.12.2020 15:00,NaN,25.12.2020 16:00,NaN,На праздничной программе гости узнают об истор...,Нет,Габричевского ул. 8,Северо-Западный административный округ,Покровское-Стрешнево,Опорно-двигательный аппарат,0,0,6.0,0,Опорно-двигательный аппарат,1,0,0,0,6,90,1.2
19623,3975,Фильм. Фильм. Фильм,ГБУК г. Москвы «ЦБС ЦАО»,викторина,культурно-досуговое,NaN,28.10.2020 16:00,NaN,28.10.2020 17:00,NaN,"""Отвечая на запрос наших читателей, среди кото...",Нет,"Факельный Б. пер. 3, строение 2",Центральный административный округ,Таганский,Опорно-двигательный аппарат,0,0,16.0,0,Опорно-двигательный аппарат,1,0,0,0,16,90,1.2
22098,6450,Шелест музыкальных страниц: креатив в камерном...,ГБУК г.Мо

In [84]:
df.to_csv('/content/drive/My Drive/Datasets_LCT/recomm.csv')

In [85]:
df.iloc[[1139, 2016, 1893], :]

,id,Название мероприятия,Наименование учреждения,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Тип ОВЗ,Стоимость от,Стоимость до,Возрастной ценз,Онлайн,ОВЗ,Опорно-двигательный аппарат,Слух,Зрение,Инвалиды-колясочники,Возраст от,Возраст до
37526,816,"Концерт проекта ""Время Ч""",ГБУК г. Москвы «ЦБС ЦАО»,"концерт, музыкальное представление",культурно-досуговое,NaN,05.11.2020,19:00,05.11.2020,21:00,"""В Чеховском культурном центре в рамках проект...",Нет,"Страстной бульв. 6, строение 2",Центральный административный округ,Тверской,Опорно-двигательный аппарат,0,0,6.0,0,Опорно-двигательный аппарат,1,0,0,0,6,60
40190,1133,Домашний уют зимних вечеров,ГБУК г. Москвы «ТКС «Планета»,"комплексное мероприятие, тематическая программа",культурно-досуговое,1 января. Новый год,17.12.2020,11:00,31.12.2020,18:00,Все студии декоративно-прикладного творчества ...,Нет,"Матвеевская ул. 18, корпус 1",Западный административный округ,Очаково-Матвеевское,"Опорно-двигательный аппарат, Слух",0,0,0.0,0,"Опорно-двигательный аппарат, Слух",1,1,0,0,8,80
39830,589,Кутузов vs Наполеон,ГБУК г. Москвы «ЦБС ЦАО»,"консультация, методическое занятие, семинар, т...",патриотическое,9 декабря. День Героев Отечества,09.12.2020,15:00,09.12.2020,17:00,"Отечественная война 1812 г.- памятное, великое...",Нет,"Смоленская пл., д.13/21",Центральный административный округ,Арбат,Опорно-двигательный аппарат,0,0,16.0,0,Опорно-двигательный аппарат,1,0,0,0,16,70
